In [ ]:
#	Prompt LLMs to be expert
#       they can now respond to question and provide structured response
#	Generate domain-specific multiple choice questions
#       on the way
#	Grade the result of LLM and find out correlations
#       with strcuture output, should be easy


In [2]:
import json
import pandas as pd
from phi.agent import Agent, RunResponse
from phi.model.ollama import Ollama
from phi.storage.agent.sqlite import SqlAgentStorage

In [85]:
# global variables
model_id = "llama3.1"
#qwen2.5-coder, llama3.1, llama3.2


In [4]:
# Prompt LLMs to be expert

# get list of occupation
occupation = pd.read_csv('occupation.txt', sep='\t').dropna(subset=["occupation_description"])
occupation = occupation[["occupation_code", "occupation_name","occupation_description"]]
occupation = occupation.rename(columns={"occupation_code":"code", "occupation_name":"name","occupation_description":"description"})


In [5]:
occupation_l = ['New Accounts Clerks', 'Nuclear Technicians', 'Orderlies', 'Clinical and Counseling Psychologists', 'Radiologic Technologists and Technicians'] #print(list(occupation.sample(5).name))
sample_o = occupation.loc[occupation["name"].isin(occupation_l)]
sample_o

,code,name,description
201,193033,Clinical and Counseling Psychologists,"Assess, diagnose, and treat mental and emotion..."
222,194051,Nuclear Technicians,"Assist nuclear physicists, nuclear engineers, ..."
469,292034,Radiologic Technologists and Technicians,Take x-rays and CAT scans or administer nonrad...
501,311132,Orderlies,Transport patients to areas such as operating ...
702,434141,New Accounts Clerks,Interview persons desiring to open accounts in...


In [72]:
#C.4 Expert Casting Prompt
#From now on, you are an excellent {role} described as {roles_description}. Answer the following question while staying in strict accordance with the nature of the provided identity: {question}.
#url = "https://arxiv.org/pdf/2411.00492"

def create_agent(role, role_description):
    expert_agent = Agent(
        model=Ollama(id=model_id),
        description="From now on, you are an excellent {} described as {}. you are tasked with answering multiple choice questions. Think through each step meticulously but only show me the final answer. Rules: 1.Think deeply about the question's core concept. 2. Consider common misconceptions. 3. Mentally justify why each wrong option is incorrect 4. Double-check your logic 5. For calculation questions, solve using two different methods if possible.".format(role, role_description),
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        instructions=["-you are helpful and you think step by step."],
        add_history_to_messages=True,
        num_history_responses=3    
    )
    return expert_agent.run("what is your occupation").content, expert_agent.session_id

In [75]:
agents_in_action = {}
for role in range(len(sample_o)):
    agents_in_action[sample_o["name"].iloc[role]] = create_agent(sample_o["name"].iloc[role], sample_o["description"].iloc[role])[1]
agents_in_action

{'Clinical and Counseling Psychologists': '39f281cb-5653-4c53-aa71-93693110ec63',
 'Nuclear Technicians': '80b2d652-2772-4d44-8c28-1775e7f8cfc4',
 'Radiologic Technologists and Technicians': 'f40c78f5-3a71-40af-b3c9-db9be14cdd06',
 'Orderlies': 'd652f109-ddf6-4c34-ba9d-6034885217e0',
 'New Accounts Clerks': 'bbd0b655-5515-4ce8-8f8c-1f83977178d8'}

In [22]:
question = "Question: A ball is thrown vertically upward with an initial velocity of 20 m/s. Assuming g = 10 m/s², what is the maximum height reached by the ball? Options: A) 10 meters B) 20 meters C) 30 meters D) 40 meters Your answer: [Option]"


In [192]:
#-you structure your output in a dictionary {"question number": "answer option"}. for example: {"1": "A", "2":"C", "3":"C"}
def ask_mc(question, session):
    expert_agent = Agent(
        model=Ollama(id=model_id),
        description="you are tasked with answering multiple choice questions. Think through each step meticulously but only show me the final answer. Rules: 1.Think deeply about the question's core concept. 2. Consider common misconceptions. 3. Mentally justify why each wrong option is incorrect 4. Double-check your logic 5. For calculation questions, solve using two different methods if possible. for output format, do this: [option (A, B, D, or D. do not repeat the answer text. alphbet only)]",
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        instructions=["""-you think step by step. -do not provide explanations. -you structure your answer in the following way: "qustionX: the answer is X". """],
        add_history_to_messages=True,
        num_history_responses=3,
        session_id= session
    )
    return expert_agent.run(question).content

In [77]:
for agent in agents_in_action:
    print(ask_mc(question, agents_in_action[agent]))

('The answer is D) Yellow.', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion1: the answer is C', '80b2d652-2772-4d44-8c28-1775e7f8cfc4')
('qustion1: the answer is C', 'f40c78f5-3a71-40af-b3c9-db9be14cdd06')
('C) Green', 'd652f109-ddf6-4c34-ba9d-6034885217e0')
('qustion1: the answer is C', 'bbd0b655-5515-4ce8-8f8c-1f83977178d8')


In [78]:
question = """Answer this multiple choice question. Think through each step meticulously but only show me the final answer.

Question: In a population of 200 students, 60% are female. How many male students are in the population?

Options:
A) 80 students
B) 120 students
C) 140 students
D) 160 students"""

In [34]:
question = """Question: Which of the following is NOT a primary color in traditional color theory?

Options:
A) Red
B) Blue
C) Green
D) Yellow"""

In [185]:
with open("questions.json", "r") as file:
    qb = json.load(file)

for q in range(len(qb)):
    print(qb[q]["question"],qb[q]["options"] )

questions = [[qb[q]["question"],qb[q]["options"]] for q in range(len(qb))]
answers = [qb[q]["answer"] for q in range(len(qb))]
questions

When positioning a patient for a chest X-ray, which adjustment ensures the clavicles are out of the lung field? ['Angle the X-ray tube cephalad.', 'Position the patient in a prone position.', 'Align the X-ray beam perpendicular to the sternum.', 'Ask the patient to roll their shoulders forward.']
What is the recommended protocol when imaging a pediatric patient who cannot remain still? ['Use sedation to keep the patient immobile.', 'Use the shortest possible exposure time and immobilization devices.', 'Allow a parent to hold the child during the procedure without protective gear.', 'Perform the imaging procedure despite movement for efficiency.']
A fluoroscopy exam shows unusually high patient skin dose readings. What should you do? ['Stop the procedure immediately and report the incident.', 'Continue the procedure but decrease the fluoroscopy time.', 'Confirm equipment calibration and assess for overexposure risks.', 'Lower the table to reduce the distance between the X-ray source and

[['When positioning a patient for a chest X-ray, which adjustment ensures the clavicles are out of the lung field?',
  ['Angle the X-ray tube cephalad.',
   'Position the patient in a prone position.',
   'Align the X-ray beam perpendicular to the sternum.',
   'Ask the patient to roll their shoulders forward.']],
 ['What is the recommended protocol when imaging a pediatric patient who cannot remain still?',
  ['Use sedation to keep the patient immobile.',
   'Use the shortest possible exposure time and immobilization devices.',
   'Allow a parent to hold the child during the procedure without protective gear.',
   'Perform the imaging procedure despite movement for efficiency.']],
 ['A fluoroscopy exam shows unusually high patient skin dose readings. What should you do?',
  ['Stop the procedure immediately and report the incident.',
   'Continue the procedure but decrease the fluoroscopy time.',
   'Confirm equipment calibration and assess for overexposure risks.',
   'Lower the table

In [167]:
prompt = """please design 10 multiple choice questions for this occupation. some thoughts before you repond: 
1. be domain specific. ideally design for someone with more than 10 year experience.
2. create some nuance. make the answers somewhat similar so that people cannot guess by intuition.
3. do not limit yourself with only factual question. be creative.
4. output your questions in json format where new line is strickly prohitbited. only produce it in compact format. sample format: 
[{"question":"When positioning a patient for a chest X-ray, which adjustment ensures the clavicles are out of the lung field?","options":["Angle the X-ray tube cephalad.","Position the patient in a prone position.","Align the X-ray beam perpendicular to the sternum.","Ask the patient to roll their shoulders forward."],"answer":"d","reason":"Rolling shoulders forward moves the clavicles out of the lung field."},{"question":"What is the recommended protocol when imaging a pediatric patient who cannot remain still?","options":["Use sedation to keep the patient immobile.","Use the shortest possible exposure time and immobilization devices.","Allow a parent to hold the child during the procedure without protective gear.","Perform the imaging procedure despite movement for efficiency."],"answer":"b","reason":"Short exposure time and immobilization reduce radiation and ensure safety."}]
5. only output the required json. your output will be directly feed into other codes so only provide the clean data and nothing else.
here is the occupation: 'Residential Plumber'"""

In [ ]:
def mc_magician(prompt):
    mc_agent = Agent(
        model=Ollama(id=model_id),
        description=""""youre someone with these qualities:
1. Deep Subject Matter Expertise
    Extensive knowledge and experience in the specific domain, ensuring that the questions test practical and theoretical knowledge relevant to the field.
    Familiarity with industry trends, tools, and challenges to create current and applicable scenarios.

2. Pedagogical Skills
    Ability to design questions that evaluate not only memorized knowledge but also problem-solving, analytical thinking, and applied skills.
    Experience in educational design or certification processes.

3. Understanding of Professional Requirements
    Awareness of the key competencies required for success in the profession. This could involve collaborating with industry professionals or studying job requirements.
    Ability to assess levels of proficiency, from beginner to expert.

4. Analytical and Research Skills
    Capable of analyzing job roles and breaking them into assessable skills and knowledge areas.
    Conducting research to stay updated on technological or methodological advancements.

5. Collaboration with Industry Practitioners
    Collaboration with professionals currently working in the field to ensure questions reflect real-world challenges and practices.
    Incorporating feedback from companies or organizations that will use the certification.

6. Access to Validation Tools
    Familiarity with tools for benchmarking and validating questions to ensure they are neither too simple nor overly complex.
    Experience with psychometric techniques to gauge question effectiveness.""",
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        instructions=["you only output what is reuqired and never provide explanation or additional information."],
        add_history_to_messages=True,
        num_history_responses=3,
        
    )
    return mc_agent.run(prompt).content, mc_agent.session_id

In [169]:
mc_magician(prompt)

('[{"question":"When performing a water meter test, what is the recommended procedure to verify accurate measurement?","options":["Check the meter\'s accuracy by comparing it to a calibrated gauge.","Verify the meter\'s installation is level and secure.","Compare the meter\'s reading with a second identical meter in series.","Calibrate the meter against a known quantity of water flow."],"answer":"c","reason":"Comparing two meters ensures accurate measurement."},{"question":"What should be done when encountered with a corroded pipe during excavation?","options":["Replace the entire section of pipe immediately.","Use a pipe cutter to remove the corroded portion only.","Tackle the issue in-house without seeking assistance from colleagues.","Seal off the area and wait for further guidance."],"answer":"b","reason":"Removing the corroded section prevents further damage."},{"question":"A homeowner reports low water pressure. Which step should be taken first?","options":["Check the main shut-o

In [186]:
result = ('[{"question":"When performing a water meter test, what is the recommended procedure to verify accurate measurement?","options":["Check the meter\'s accuracy by comparing it to a calibrated gauge.","Verify the meter\'s installation is level and secure.","Compare the meter\'s reading with a second identical meter in series.","Calibrate the meter against a known quantity of water flow."],"answer":"c","reason":"Comparing two meters ensures accurate measurement."},{"question":"What should be done when encountered with a corroded pipe during excavation?","options":["Replace the entire section of pipe immediately.","Use a pipe cutter to remove the corroded portion only.","Tackle the issue in-house without seeking assistance from colleagues.","Seal off the area and wait for further guidance."],"answer":"b","reason":"Removing the corroded section prevents further damage."},{"question":"A homeowner reports low water pressure. Which step should be taken first?","options":["Check the main shut-off valve\'s functionality.","Verify the household plumbing layout is correct.","Run a diagnostic test on the entire system.","Clear any debris from the meter or pipes."],"answer":"a","reason":"Checking the shut-off valve ensures no leaks are causing low pressure."},{"question":"What safety precaution should be taken when working with chemicals in confined spaces?","options":["Wear protective gloves and eyewear only.","Ensure good ventilation to prevent inhalation of fumes.","Use a respirator when handling hazardous materials.","Take regular breaks to stretch outside the work area."],"answer":"b","reason":"Good ventilation prevents chemical inhalation hazards."},{"question":"A homeowner requests installation of a water-efficient showerhead. What should be done?","options":["Recommend upgrading to a tankless water heater instead.","Verify the existing plumbing can accommodate the new device.","Ensure compliance with local regulations and building codes.","Install the product without inspecting for compatibility issues."],"answer":"b","reason":"Ensuring compatibility prevents future plumbing problems."},{"question":"When working with flexible PEX tubing, what is the recommended method to prevent damage?","options":["Support the tubing with clamps at regular intervals.","Use a pipe cutter to remove excess material only when necessary.","Route the tubing along existing structural elements for added support.","Run the tubing through a protective sleeve for added protection."],"answer":"d","reason":"Using a protective sleeve prevents damage and kinking."},{"question":"A residential plumber encounters an issue with a sewage backup into a home. What immediate action should be taken?","options":["Call the homeowner to discuss mitigation strategies.","Contact local authorities for assistance in resolving the issue.","Seal off the affected area immediately to prevent further contamination.","Clear any debris or obstructions blocking the sewage line."],"answer":"c","reason":"Sealing off prevents further damage and contamination."},{"question":"What should be done when dealing with a frozen pipe during the winter season?","options":["Apply heat directly to the affected section using an open flame.","Use insulation materials to prevent re-freezing.","Tackle the issue in-house without seeking assistance from colleagues.","Call a professional plumber for assistance as soon as possible."],"answer":"d","reason":"Calling a professional ensures proper handling and prevention of future issues."},{"question":"A homeowner requests installation of a sump pump system. What should be done first?","options":["Verify local regulations regarding sump pump installations.","Design the system with consideration to drainage patterns and existing piping.","Check for any electrical or plumbing code violations in the area.","Clear debris from the yard before beginning installation."],"answer":"b","reason":"Considering drainage prevents future problems and ensures proper functioning."},{"question":"What should be done when encountered with asbestos-containing materials during excavation?","options":["Leave the material undisturbed to avoid exposure.","Use protective gear and remove the material carefully.","Tackle the issue in-house without seeking assistance from colleagues.","Seal off the area immediately and wait for further guidance."],"answer":"b","reason":"Careful removal prevents exposure hazards."}]',
 '27fcd85f-3dc8-461f-9403-b6c30125d4be')
qb = json.loads(result[0])
for q in range(len(qb)):
    print(qb[q]["question"],qb[q]["options"] )

questions = [[qb[q]["question"],qb[q]["options"]] for q in range(len(qb))]
answers = [qb[q]["answer"] for q in range(len(qb))]
questions

When performing a water meter test, what is the recommended procedure to verify accurate measurement? ["Check the meter's accuracy by comparing it to a calibrated gauge.", "Verify the meter's installation is level and secure.", "Compare the meter's reading with a second identical meter in series.", 'Calibrate the meter against a known quantity of water flow.']
What should be done when encountered with a corroded pipe during excavation? ['Replace the entire section of pipe immediately.', 'Use a pipe cutter to remove the corroded portion only.', 'Tackle the issue in-house without seeking assistance from colleagues.', 'Seal off the area and wait for further guidance.']
A homeowner reports low water pressure. Which step should be taken first? ["Check the main shut-off valve's functionality.", 'Verify the household plumbing layout is correct.', 'Run a diagnostic test on the entire system.', 'Clear any debris from the meter or pipes.']
What safety precaution should be taken when working with

[['When performing a water meter test, what is the recommended procedure to verify accurate measurement?',
  ["Check the meter's accuracy by comparing it to a calibrated gauge.",
   "Verify the meter's installation is level and secure.",
   "Compare the meter's reading with a second identical meter in series.",
   'Calibrate the meter against a known quantity of water flow.']],
 ['What should be done when encountered with a corroded pipe during excavation?',
  ['Replace the entire section of pipe immediately.',
   'Use a pipe cutter to remove the corroded portion only.',
   'Tackle the issue in-house without seeking assistance from colleagues.',
   'Seal off the area and wait for further guidance.']],
 ['A homeowner reports low water pressure. Which step should be taken first?',
  ["Check the main shut-off valve's functionality.",
   'Verify the household plumbing layout is correct.',
   'Run a diagnostic test on the entire system.',
   'Clear any debris from the meter or pipes.']],
 [

In [189]:
answers

['c', 'b', 'a', 'b', 'b', 'd', 'c', 'd', 'b', 'b']

In [ ]:
def answer_cleaner(input):
    clean_agent = Agent(
        model=Ollama(id=model_id),
        description="you are a profession computer programmer with 20 years of experience. you are good at cleaning data and cextract answer to be a list of strings. you will receive input, and please extract answer from it. your output will be directly feed into other codes so only provide the clean data and nothing else.example: ['x','x','x','x','x','x'] where 'x' is either D, B, C, or A",
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        instructions=["you only output what is reuqired and never provide explanation or additional information"],
        add_history_to_messages=True,
        num_history_responses=3,
        
    )
    return clean_agent.run(input).content

In [195]:
for agent in agents_in_action:
    store = []
    for q in range(len(questions)):
        mess = (ask_mc("""format is very important. remember to structure your result. no need to provide reaons. here is the question:"""+ str(questions[q]), agents_in_action[agent]))
        store.append(mess)
    clean = answer_cleaner(store)
    print(clean, agents_in_action[agent])


KeyboardInterrupt: 

In [190]:
store

[('', '7b0ea7be-49be-480f-a810-95a529125472'),
 ('', '1f601db4-7f71-424e-87a8-a96a9aa377bd'),
 ('', '1a50066f-ee7c-4796-b8fe-8cae71f1c902'),
 ('', '70be8b5a-5898-4d71-a3ee-eb2bac0efc2a'),
 ('', '574971a1-d045-4c91-ba3e-570d115f9149'),
 ('', 'ab793c5e-a807-4dd0-8741-37a051234e74'),
 ('', '6e4a4542-1d18-4d69-b105-528967ca9a3c'),
 ('', '34f921f5-fb68-417d-b070-4db079d67982'),
 ('', 'fc11e6fe-7793-4582-9c0c-b207d35d1699'),
 ('', '4d899baa-2aff-4eaf-ad6f-7291615b01a0'),
 ('', 'fe6c81c3-9e50-4360-b052-2605a7d00fb5'),
 ('', 'f8a1bf45-77b9-45da-953a-623b25943a41'),
 ('', '7ef9f6de-1874-4c0b-86ce-254016bc83c6'),
 ('', '41a45682-7b44-47bd-8cc8-d12224d0fa79'),
 ('', '010500fa-c6f4-4d5f-8066-7aa0507a2508'),
 ('', 'fe4b9e27-e576-46c5-8a94-13280988aa37'),
 ('', '66d3297a-b3b2-4654-8b25-fe4bfe7b4bba'),
 ('', 'b2c1a450-3ce7-4208-b184-983c59cb3ee5'),
 ('', '41e50342-623c-4ba4-962b-215670462f50'),
 ('', '5ce98f1e-890e-4d68-bba5-0fc8e509ff29'),
 ('', '48c8077a-9804-4c04-bec7-9d623d694734'),
 ('', '377753

In [107]:
messydata = """('qustion:1, answer:A', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:2, answer:B', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:3, answer:C', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:4, answer:C', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:5, answer:C', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:6, answer:B', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:7, answer:C', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:8, answer:D', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:9, answer:A', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:10, answer:B', '39f281cb-5653-4c53-aa71-93693110ec63')"""

In [121]:
eval(answer_cleaner(messydata)[0])

['A', 'B', 'C', 'C', 'C', 'C', 'B', 'C', 'D', 'A', 'B']